1/5000国土基本図郭のポリゴンを作成する
on 1 Sep. 2021 by Hiromu Daimaru

In [1]:
# ライブラリをインポートする
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
import fiona
# from fiona.crs import from_epsg

In [2]:
# データ収納用の空のデータフレームを作成する
dfMesh = gpd.GeoDataFrame()
dfMesh['geometry'] = None
dfMesh['code'] = None

In [3]:
# 平面直角座標系（19座標系）の番号を指定
code19 = 10

国土基本図図郭の命名ルールについては下記を参照
空間情報クラブ
https://club.informatix.co.jp/?p=1293

In [4]:
# 1/50000図郭名のＸ成分、Ｙ成分の配列を作成する
x_code50k = [chr(i) for i in range(ord('A'), ord('H')+1)]
y_code50k = [chr(i) for i in range(ord('A'),ord('T')+1  )]
# Yの順を逆にする（高緯度－＞低緯度の順なので）
y_code50k.reverse()

In [5]:
# 1/50000図郭の座標値の配列を作成する
x_loc50k = [i * 40000 for i in range(-4, 4+1)]
y_loc50k = [i * 30000 for i in range(-10, 10 + 1)]

In [6]:
# 1/5000図郭の座標値の配列を作成する
x_loc5k = [i * 4000 for i in range(0, 10 + 1)]
y_loc5k = [i * -3000 for i in range(0, 10 + 1)]

In [7]:
id = 0
for y50k in range(len(y_code50k)):
    for x50k in range(len(x_code50k)):
        code50k = y_code50k[y50k] + x_code50k[x50k] 
        x_min50k = x_loc50k[x50k]
        y_min50k = y_loc50k[y50k]
        x_max50k = x_loc50k[x50k + 1]
        y_max50k = y_loc50k[y50k + 1]
        coordinates50k = ((x_min50k, y_min50k), (x_min50k, y_max50k), (x_max50k, y_max50k), (x_max50k, y_min50k), (x_min50k, y_min50k))
        for y5k in range(10):
            y_max5k = y_max50k + y_loc5k[y5k]
            y_min5k = y_max50k + y_loc5k[y5k+1]
            for x5k in range(10):
                x_min5k = x_min50k + x_loc5k[x5k]
                x_max5k = x_min50k + x_loc5k[x5k+1]                
                coordinates5k = ((x_min5k, y_min5k), (x_min5k, y_max5k), (x_max5k, y_max5k), (x_max5k, y_min5k), (x_min5k, y_min5k))                 
                poly = Polygon(coordinates5k)
                code5k = str(y5k) + str(x5k)
                code = str(code19).zfill(2) + code50k + code5k
                dfMesh.loc[id, 'geometry'] = poly
                dfMesh.loc[id, 'code'] = code
                id += 1

In [8]:
dfMesh.head()

,geometry,code
0,"POLYGON ((-160000.000 -273000.000, -160000.000...",10TA00
1,"POLYGON ((-156000.000 -273000.000, -156000.000...",10TA01
2,"POLYGON ((-152000.000 -273000.000, -152000.000...",10TA02
3,"POLYGON ((-148000.000 -273000.000, -148000.000...",10TA03
4,"POLYGON ((-144000.000 -273000.000, -144000.000...",10TA04


In [9]:
# 座標系を平面直角座標系（JGD2011）に設定
# 19座標系のEPSGコードは下記を参照
# https://tmizu23.hatenablog.com/entry/20091215/1260868350
from fiona.crs import from_epsg
dfMesh.crs = from_epsg(6668 + code19)
# dfMesh.crs

C:\Users\hirom\Anaconda3\envs\geopandas\lib\site-packages\pyproj\crs\crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [10]:
dfMesh['area'] = dfMesh['geometry'].area
dfMesh.head()

,geometry,code,area
0,"POLYGON ((-160000.000 -273000.000, -160000.000...",10TA00,12000000.0
1,"POLYGON ((-156000.000 -273000.000, -156000.000...",10TA01,12000000.0
2,"POLYGON ((-152000.000 -273000.000, -152000.000...",10TA02,12000000.0
3,"POLYGON ((-148000.000 -273000.000, -148000.000...",10TA03,12000000.0
4,"POLYGON ((-144000.000 -273000.000, -144000.000...",10TA04,12000000.0


In [11]:
# WGS1984（緯度経度）に投影変換
dfMesh=dfMesh.to_crs(epsg=4326)

In [12]:
dfMesh.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [13]:
dfMesh.head()

,geometry,code,area
0,"POLYGON ((139.02315 37.52667, 139.02250 37.553...",10TA00,12000000.0
1,"POLYGON ((139.06839 37.52735, 139.06775 37.554...",10TA01,12000000.0
2,"POLYGON ((139.11362 37.52802, 139.11300 37.555...",10TA02,12000000.0
3,"POLYGON ((139.15886 37.52867, 139.15826 37.555...",10TA03,12000000.0
4,"POLYGON ((139.20410 37.52930, 139.20351 37.556...",10TA04,12000000.0


In [14]:
#シェープファイル名を指定して出力　下記の右辺は任意のパスに変更してください
outfp = r"shapefile\zukaku_mesh_XY"+ str(code19)+ r".shp"
dfMesh.to_file(outfp)

In [15]:
# jsonファイル名を指定して出力
# outfp = r"C:\Users\hirom\Dropbox\learning\udemy_geopandas\geopandas\zukaku\geojson\zukaku_mesh_"+ str(code19)+ r".geojson"
# dfMesh.to_file(outfp, driver="GeoJSON")